In [102]:
import pandas as pd
import json
from math import radians, cos, sin, asin, sqrt
import math
import numpy as np
from geopy.geocoders import Nominatim

In [113]:
# import the data in a dataframe
json_data=open('nuclear_tests.json').read()
data = json.loads(json_data)
df = pd.DataFrame(data)
df.head()
print(len(df))

2065


In [114]:
print(str(len(df[pd.isnull(df['decimal latitude/longitude'])])) + ' / ' + str(len(df)) + ' entries do not have coordinates')
print(str(len(df[df['site'] == ''])) + ' / ' + str(len(df)) + ' entries do not have a site')
distinct_sites = set(df['site'])
print('There are ' + str(len(distinct_sites)) + ' distinct sites')

1211 / 2065 entries do not have coordinates
119 / 2065 entries do not have a site
There are 36 distinct sites


In [115]:
# Only one entry has neither a site nor coordinates:
df[(pd.isnull(df['decimal latitude/longitude'])) & (df['site'] == '')]

body wave magnitude country        date decimal latitude/longitude name  \
2064                 NaN     UNK  22/09/1979                        NaN    -   

                              original time site         type yield  
2064  03:00:00 (Universal Coordinated Time)       Atmospheric   NaN

In [111]:
# remove null site/location from dataframe
df = df[(pd.notnull(df['decimal latitude/longitude'])) | (df['site'] != '')]

In [121]:
# We group the dataframe by sites and export the coordinates of each event in a distinct list for each site
# the goal is to see events on a same site can have very different coordinates or not.
coord_lists = df.fillna('-').groupby('site')['decimal latitude/longitude'].apply(list)

In [122]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [140]:
no_coord_sites = []
for i in range(0, len(coord_lists)):
    dists = []
    lat0 = ""
    long0 = ""
    print(coord_lists.index[i])
    for event in coord_lists[i]:
        if('/' in str(event)):
            lat0, long0 = event.split('/')
            break
    if (lat0!="" and long0!=""):
        for event in coord_lists[i]:
            event = str(event)
            if('/' in event):
                lat, long = event.split('/')
                dists.append(haversine(float(long0), float(lat0), float(long), float(lat)))

        
        print(np.mean(dists))
    else:
        no_coord_sites.append(coord_lists.index[i])
        print("No distances")
    print("---------")



1651.56727716
---------
Amchitka, Alaska
4.78866535241
---------
Bikini Is (USA)
32.7202064536
---------
Carlsnad, New Mexico (USA)
No distances
---------
Central Nevada
No distances
---------
Christmas Island
No distances
---------
Christmas Island (USA)
No distances
---------
Ekker, Algeria
1.42693253151
---------
Emu Fields, Australia
No distances
---------
Enewetak (USA)
53.7383584401
---------
Fallon, Nevada
No distances
---------
Farmington, New Mexico
No distances
---------
Grand Valley, Colorado
No distances
---------
Hattiesburg, Mississippi
No distances
---------
Hiroshima, Japan
0.0
---------
Johnston Island
No distances
---------
Kharan
0.0
---------
Lop Nor
54.0866798425
---------
Maralinga
No distances
---------
Missile Testing Range
138.40896036
---------
Monte Bello Is
No distances
---------
NTS
No distances
---------
Nagasaki, Japan
0.0
---------
Nellis Air Force Range
No distances
---------
Nevada Test Site
5.55569994183
---------
New Mexico (USA)
No distances
------

In [ ]:
# As we can see, some events have a site but no location. 

In [103]:
geolocator = Nominatim()

print(location.latitude)
print(location.longitude)

-10.4909589
105.617414292465


In [141]:
#first make a dict site => coords (from geopy)
site_coords = dict()
for site in no_coord_sites:
    location = geolocator.geocode(site)
    if location is not None:
        coord = str(location.latitude) + "/" + str(location.longitude)
        site_coords[site] = coord
    else:
        print("coordinates not found for site " + str(site))
#site_coords

coordinates not found for site Carlsnad, New Mexico (USA)
coordinates not found for site Emu Fields, Australia
coordinates not found for site Nellis Air Force Range


In [142]:
print(len(no_coord_sites))
print(len(site_coords))

19
16


In [162]:
#df[df['site'] in site_coords.keys()]['decimal latitude/longitude'] = site_coords[df['site']]
df[df['site'].isin(list(site_coords.keys()))]['decimal latitude/longitude'] = df[df['site'].isin(list(site_coords.keys()))]['site'].map(site_coords)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [171]:
for i in df.index:
    if('/' not in str(df['decimal latitude/longitude'][i])):
        if (df['site'][i] in site_coords.keys()):
            df['decimal latitude/longitude'][i] = site_coords[df['site'][i]]

In [172]:
df[df['site'].isin(list(site_coords.keys()))]

body wave magnitude country        date   decimal latitude/longitude  \
45                   5.3     FRA  27/01/1996     -18.7963088/-141.5839778   
46                   5.1     FRA  27/12/1995     -18.7963088/-141.5839778   
47                   4.8     FRA  21/11/1995     -18.7963088/-141.5839778   
48                   5.4     FRA  27/10/1995     -18.7963088/-141.5839778   
49                   5.4     FRA  01/10/1995     -18.7963088/-141.5839778   
50                   4.8     FRA  05/09/1995     -18.7963088/-141.5839778   
51                   5.3     FRA  15/07/1991     -18.7963088/-141.5839778   
52                   3.8     FRA  05/07/1991     -18.7963088/-141.5839778   
53                   5.2     FRA  14/06/1991     -18.7963088/-141.5839778   
54                   5.5     FRA  29/05/1991     -18.7963088/-141.5839778   
55                   5.1     FRA  18/05/1991     -18.7963088/-141.5839778   
56                   NaN     FRA  07/05/1991     -18.7963088/-141.5839778   
57                   5.4     FRA  21/11/1990     -18.7963088/-141.5839778   
58                   5.5     FRA  14/11/1990     -18.7963088/-141.5839778   
59                   5.1     FRA  04/07/1990     -18.7963088/-141.5839778   
60                   5.5     FRA  26/06/1990     -18.7963088/-141.5839778   
61                  4.65     FRA  07/06/1990     -18.7963088/-141.5839778   
62                   5.3     FRA  02/06/1990     -18.7963088/-141.5839778   
63                  5.59     FRA  27/11/1989     -18.7963088/-141.5839778   
64                  5.19     FRA  20/11/1989     -18.7963088/-141.5839778   
65                   5.3     FRA  31/10/1989     -18.7963088/-141.5839778   
66                  5.37     FRA  24/10/1989     -18.7963088/-141.5839778   
67                  5.52     FRA  10/06/1989     -18.7963088/-141.5839778   
68                  5.16     FRA  03/06/1989     -18.7963088/-141.5839778   
69                   NaN     FRA  20/05/1989     -18.7963088/-141.5839778   
70                  5.16     FRA  11/05/1989     -18.7963088/-141.5839778   
71                  5.58     FRA  30/11/1988     -18.7963088/-141.5839778   
72                  5.29     FRA  23/11/1988     -18.7963088/-141.5839778   
73                   5.3     FRA  05/11/1988     -18.7963088/-141.5839778   
74                   NaN     FRA  25/10/1988     -18.7963088/-141.5839778   
...                  ...     ...         ...                          ...   
1795                 NaN     USA  10/07/1962       27.8405916/-82.5941972   
1796                 NaN     USA  09/07/1962  16.728912/-169.533849659687   
1800                 NaN     USA  30/06/1962       27.8405916/-82.5941972   
1803                 NaN     USA  27/06/1962       27.8405916/-82.5941972   
1804                 NaN     USA  22/06/1962       27.8405916/-82.5941972   
1806                 NaN     USA  19/06/1962       27.8405916/-82.5941972   
1807                 NaN     USA  17/06/1962       27.8405916/-82.5941972   
1808                 NaN     USA  15/06/1962       27.8405916/-82.5941972   
1810                 NaN     USA  12/06/1962       27.8405916/-82.5941972   
1811                 NaN     USA  10/06/1962       27.8405916/-82.5941972   
1812                 NaN     USA  09/06/1962       27.8405916/-82.5941972   
1813                 NaN     USA  08/06/1962       27.8405916/-82.5941972   
1816                 NaN     USA  27/05/1962       27.8405916/-82.5941972   
1818                 NaN     USA  25/05/1962       27.8405916/-82.5941972   
1820                 NaN     USA  19/05/1962       27.8405916/-82.5941972   
1821                 NaN     USA  14/05/1962       27.8405916/-82.5941972   
1823                 NaN     USA  12/05/1962       27.8405916/-82.5941972   
1825                 NaN     USA  11/05/1962       27.8405916/-82.5941972   
1827                 NaN     USA  09/05/1962       27.8405916/-82.5941972   
1828                 NaN     USA  08/05/1962       27.8405916/-82.5941972   
1